In [1]:
import pandas as pd
import yfinance as yf
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
from psycopg2.errors import UniqueViolation
from fin_func import *

In [2]:
# Database connection details
database = 'fin_data'
user = 'stevengiallourakis'
password = 'Birdman11!' # this might not be needed depenending on where you are connecting from.
host = 'localhost'
port =  '5433'
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [ ]:
# DB = stocks, table = stocks

stock_data = get_stock_data(symbol, start_date, end_date)

# add data to database - table stocks

data_to_db(stock_data, 'stocks', engine)

print(stock_data)

In [6]:
# Details for commodities api request

#symbol = ''
start_date = '2020-01-01'
end_date = '2024-07-04'

In [ ]:
# DB = stocks, table = commodities
commodities_data = get_commodities_data(symbol, start_date, end_date)

data_to_db(commodities_data, 'commodities', engine)

print(commodities_data)

In [ ]:
# This cell contains old code for bulk download

# Bulk Data download



#bulk_data = []

# Iterate through list of stocks

#for symbol in bulk_data:
      #  print("Retrieving stock data")
            
        # Retrieve stock data and update dataframe to fit database
      #   stock_data = get_stock_data(symbol, start_date, end_date)


        # Check if the stock data is successfully downloaded
      #  if stock_data.empty:
       #     print(f"Failed to download data for {symbol}")
      #  else:
       #     print(f"Stock data downloaded and updated for {symbol}")

    
        # Send data to database
       # data_to_db(daily_stock_prices, 'stocks', engine)

In [3]:
# This code is designed to create a list of companies by letter.
# This is being done so that I can break up the size of the data downloads from yfinance.
# I can designate the letter i want to start with and the code will create a list for companies
# that start with that letter.

def create_company_list_by_letter(letter, engine):

    # Connect to the database
    with engine.connect() as connection:
        # Query to select companies where the name starts with the specified letter
        result = connection.execute(
            text("SELECT stock_symbol, company_name FROM company_list WHERE company_name LIKE :letter"),
            {"letter": f"{letter}%"}
        ).fetchall()

    # Convert result to a list of dictionaries
    companies = [{"stock_symbol": row[0], "company_name": row[1]} for row in result]
    return companies



In [46]:
letter = 'Z'
# Letters downloaded and added to database
# Completed: A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z  
# Dates Checked: ''2010-01-01' - '2024-07-01'
companies = create_company_list_by_letter(letter, engine)
print(f"Companies starting with {letter}:")
for company in companies:
    print(company)

Companies starting with Z:
{'stock_symbol': 'ZA', 'company_name': 'Zuoan Fashion Limited American'}
{'stock_symbol': 'ZB-A', 'company_name': 'Zions Bc Dp Shs A'}
{'stock_symbol': 'ZB-C', 'company_name': 'Zions Bancorporation'}
{'stock_symbol': 'ZB-F', 'company_name': 'Zions Bancorporation'}
{'stock_symbol': 'ZB-G', 'company_name': 'Zions Bancorporation'}
{'stock_symbol': 'ZB-H', 'company_name': 'Zions Bancorporation'}
{'stock_symbol': 'ZEP', 'company_name': 'Zep Inc.'}
{'stock_symbol': 'ZF', 'company_name': 'Zweig Fund'}
{'stock_symbol': 'ZFC', 'company_name': 'Zais Financial Corp. Common Sto'}
{'stock_symbol': 'ZLC', 'company_name': 'Zale Corporation'}
{'stock_symbol': 'ZMH', 'company_name': 'Zimmer Holdings'}
{'stock_symbol': 'ZTR', 'company_name': 'Zweig Total Return Fund'}
{'stock_symbol': 'ZTS', 'company_name': 'Zoetis Inc. Class A Common Stoc'}
{'stock_symbol': 'Z', 'company_name': 'Zillow Inc. Cl A'}
{'stock_symbol': 'ZAGG', 'company_name': 'Zagg Inc.'}
{'stock_symbol': 'ZAZA', 

In [47]:
def get_stock_data(stock_symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    if not isinstance(stock_symbol, str):
        raise TypeError("'stock_symbol' must be a string representing a valid stock symbol")
    try:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    except Exception as e:
        logging.error(f"Error converting dates to datetime: {e}")
        return None
    try:
        stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    except Exception as e:
        logging.error(f"Error downloading stock data: {e}")
        return None

    if stock_data.empty:
        logging.error(f"Error stock data received. Aborting further operations.")
        return None

    # Drop unnecessary columns
    stock_data = stock_data.drop(columns=['High', 'Low', 'Adj Close', 'Volume'], errors='ignore')

    # Calculate daily change and percent change
    stock_data['daily_change'] = stock_data['Close'] - stock_data['Open']
    stock_data['percent_change'] = (
            ((stock_data['Close'] - stock_data['Open']) / stock_data['Open']) * 100)

    # Add the stock symbol column
    stock_data['stock_symbol'] = stock_symbol

    
    stock_data.reset_index(inplace=True)
    logging.info("Stock_data downloaded, High and Low columns removed, daily_change added")
    return stock_data



def data_to_db(df, table_name, engine):
    try:
        with engine.begin() as connection:
            for index, row in df.iterrows():
                sql = text(f"""
                    INSERT INTO {table_name} ("Date", "Open", "Close", daily_change, percent_change, stock_symbol)
                    VALUES (:Date, :Open, :Close, :daily_change, :percent_change, :stock_symbol)
                    ON CONFLICT (stock_symbol, "Date") DO NOTHING
                """)
                connection.execute(sql, {
                    'Date': row['Date'],
                    'Open': row['Open'],
                    'Close': row['Close'],
                    'daily_change': row['daily_change'],
                    'percent_change': row['percent_change'],
                    'stock_symbol': row['stock_symbol']
                })
        

        
        #df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        #print("Data successfully added to the database.")
    
    except UniqueViolation as e:
        print(f"Duplicate entry found for {stock_symbol} on {data['date']}. Skipping insert.")
    except Exception as e:
        print(f"An error occurred while inserting data: {e}")
    

In [ ]:
print(stock_data)

In [48]:
from tqdm import tqdm
import csv

# File paths for output CSVs
success_file = 'success_companies.csv'
failure_file = 'failed_companies.csv'

# Initialize lists to store successful and failed companies
success_companies = []
failed_companies = []

# Process companies with progress bar
for company in tqdm(companies, desc='Processing Companies'):
    stock_symbol = company["stock_symbol"]
    stock_data = get_stock_data(stock_symbol, start_date, end_date)
    if stock_data is not None:
        try:
            data_to_db(stock_data, 'daily_stock_prices', engine)
            success_companies.append(stock_symbol)
        except Exception as e:
            failed_companies.append(stock_symbol)
            print(f"Error adding {stock_symbol} to database: {e}")
    else:
        failed_companies.append(stock_symbol)
        print(f"Failed to retrieve data for {stock_symbol}")

# Write successful companies to CSV
#with open(success_file, 'w', newline='') as f:
 #   writer = csv.writer(f)
 #   writer.writerow(['Stock Symbol'])
 #   for stock_symbol in success_companies:
  #      writer.writerow([stock_symbol])

# Write failed companies to CSV
#with open(failure_file, 'w', newline='') as f:
 #   writer = csv.writer(f)
 #   writer.writerow(['Stock Symbol'])
 #   for stock_symbol in failed_companies:
  #      writer.writerow([stock_symbol])

#print("Processing complete.")

[*********************100%%**********************]  1 of 1 completed00<?, ?it/s]
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZA']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-01-01 00:00:00 -> 2024-07-04 00:00:00)')
ERROR:root:Error stock data received. Aborting further operations.
Processing Companies:   3%|▋                     | 1/32 [00:00<00:04,  6.56it/s]

$ZA: possibly delisted; No price data found  (1d 2020-01-01 00:00:00 -> 2024-07-04 00:00:00)
Failed to retrieve data for ZA


[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZB-A']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
ERROR:root:Error stock data received. Aborting further operations.
Processing Companies:   6%|█▍                    | 2/32 [00:00<00:14,  2.01it/s]

Failed to retrieve data for ZB-A


[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZB-C']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
ERROR:root:Error stock data received. Aborting further operations.
Processing Companies:   9%|██                    | 3/32 [00:01<00:18,  1.55it/s]

...

[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZNGA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
ERROR:root:Error stock data received. Aborting further operations.
[*********************100%%**********************]  1 of 1 completed,  1.67it/s]
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZOLT']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-01-01 00:00:00 -> 2024-07-04 00:00:00)')
ERROR:root:Error stock data received. Aborting further operations.
Processing Companies:  94%|███████████████████▋ | 30/32 [00:14<00:00,  2.15it/s]

Failed to retrieve data for ZNGA
$ZOLT: possibly delisted; No price data found  (1d 2020-01-01 00:00:00 -> 2024-07-04 00:00:00)
Failed to retrieve data for ZOLT


[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZOOM']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
ERROR:root:Error stock data received. Aborting further operations.
Processing Companies:  97%|████████████████████▎| 31/32 [00:15<00:00,  1.83it/s]

Failed to retrieve data for ZOOM


[*********************100%%**********************]  1 of 1 completed
Processing Companies: 100%|█████████████████████| 32/32 [00:16<00:00,  1.99it/s]


In [23]:
print(success_companies)


['ASGN', 'IX', 'OB', 'OC', 'OCN', 'ODC', 'ODP', 'OFG', 'OGE', 'OHI', 'OI', 'OII', 'OILT', 'OIS', 'OKE', 'OLN', 'OLP', 'OMC', 'OMI', 'ONB', 'OPK', 'OPY', 'ORA', 'ORAN', 'ORCL', 'ORI', 'ORN', 'OSK', 'OXM', 'OXY', 'GEOS', 'OCC', 'OCFC', 'ODFL', 'OFED', 'OFIX', 'OFLX', 'OFS', 'OMCL', 'OMER', 'OMEX', 'ONCY', 'OPEN', 'OPHC', 'OPOF', 'OPTT', 'ORBT', 'ORLY', 'ORMP', 'ORRF', 'OSBC', 'OSIS', 'OSUR', 'OTEX', 'OTTR', 'OVBC', 'OVLY', 'OXLC', 'OXLCO', 'OXLCP', 'SEED', 'ZEUS']
